In [ ]:
# how big is radial grid?
# one grid for all atoms?

In [1]:
import sys 
sys.path.insert(0,'/home/misa/git_repositories/APDFT/prototyping/atomic_energies')
import alchemy_tools2 as at
import alchemy_tools_pyscf as atp

from explore_qml_data import shift2center
from parse_pyscf import read_input

import pyscf.dft
import os
import qml
import numpy as np
import ase.units

In [2]:
run_dir = '/home/misa/projects/Atomic-Energies/data/vacuum_reference/qm9_data/dsgdb9nsd_001212'  #os.getcwd()
basis = 'def2-qzvp'
com = qml.Compound(xyz=os.path.join(run_dir, 'dsgdb9nsd_001212.xyz'))
coords_shifted = shift2center(com.coordinates, np.array([10.0, 10, 10]))
com.coordinates = coords_shifted

In [3]:
num_elec = atp.get_num_elec(1.0, com.nuclear_charges.sum())
deltaZ, includeonly, mol = atp.prepare_input(com.coordinates, com.nuclear_charges, num_elec, basis)

In [4]:
grid = pyscf.dft.gen_grid.Grids(mol)
grid.level = 8
grid.build()

#### Size of grid

In [5]:
min_max = dict()
for i in range(3):
    min_max[i] = (np.amin(grid.coords[:,i]), np.amax(grid.coords[:,i]))

In [6]:
min_max

{0: (-6.23153934849012, 42.31071464666762),
 1: (-5.841047830568526, 42.75118229745031),
 2: (-1.1032326571954876, 38.88660708691812)}

#### Get grid points in unit cell

In [7]:
idc_x = np.where((0<grid.coords[:,0]) & (grid.coords[:,0]<20/ase.units.Bohr))
idc_y = np.where((0<grid.coords[:,1]) & (grid.coords[:,1]<20/ase.units.Bohr))
idc_z = np.where((0<grid.coords[:,2]) & (grid.coords[:,2]<20/ase.units.Bohr))

intersection_xy = np.intersect1d(idc_x, idc_y)
intersection = np.intersect1d(intersection_xy, idc_z)

In [8]:
grid_coords_box = grid.coords[intersection]

In [9]:
min_max_box = dict()
for i in range(3):
    min_max_box[i] = (np.amin(grid_coords_box[:,i]), np.amax(grid_coords_box[:,i]))

In [10]:
min_max_box

{0: (0.012973391117558464, 37.793849478117515),
 1: (0.027765931197146543, 37.78478136156957),
 2: (0.1548256840893778, 37.63695150808833)}

#### wrap up in function and compare result to above

In [11]:
bp = [[0, 20/ase.units.Bohr], [0, 20/ase.units.Bohr], [0, 20/ase.units.Bohr]]

In [12]:
idc = at.get_idc_bl_grid_in_box(grid.coords, bp)

In [13]:
np.array_equal(intersection, idc)

True

In [16]:
grid_coords_box2 = grid.coords[idc]

In [17]:
min_max_box2 = dict()
for i in range(3):
    min_max_box2[i] = (np.amin(grid_coords_box2[:,i]), np.amax(grid_coords_box2[:,i]))

In [18]:
min_max_box2

{0: (0.012973391117558464, 37.793849478117515),
 1: (0.027765931197146543, 37.78478136156957),
 2: (0.1548256840893778, 37.63695150808833)}